In [1]:
! wget https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-03-18 01:13:44--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.124, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1710983624&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDk4MzYyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhY

In [10]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [25]:
from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
from datasets import load_dataset

data = load_dataset("b-mc2/sql-create-context")
input_template = """Generate valid SQL query for a given natural language query and schema.
In your response only provide valid SQL for the schema, do not provide anything else.

Natural language query:
{question}

Schema:
```sql
{schema}
```
"""

output_teplate = """## RESPONSE
```sql
{answer}
```
"""

def prompt(data):
    input_text = input_template.format(question=data["question"], schema=data["context"])
    output_text = output_teplate.format(answer=data["answer"])
    return {'input': input_text, "output": output_text}

data = data.map(prompt).remove_columns(["question", "answer", "context"])
data["train"].to_json("nsql.jsonl")
data

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 78577
    })
})

In [26]:
! head nsql.jsonl

{"input":"Generate valid SQL query for a given natural language query and schema.\nIn your response only provide valid SQL for the schema, do not provide anything else.\n\nNatural language query:\nHow many heads of the departments are older than 56 ?\n\nSchema:\n```sql\nCREATE TABLE head (age INTEGER)\n```\n","output":"## RESPONSE\n```sql\nSELECT COUNT(*) FROM head WHERE age > 56\n```\n"}
{"input":"Generate valid SQL query for a given natural language query and schema.\nIn your response only provide valid SQL for the schema, do not provide anything else.\n\nNatural language query:\nList the name, born state and age of the heads of departments ordered by age.\n\nSchema:\n```sql\nCREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)\n```\n","output":"## RESPONSE\n```sql\nSELECT name, born_state, age FROM head ORDER BY age\n```\n"}
{"input":"Generate valid SQL query for a given natural language query and schema.\nIn your response only provide valid SQL for the schema, do not p

In [28]:
! nvidia-smi

Mon Mar 18 04:37:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          Off | 00000000:04:00.0 Off |                    0 |
| N/A   33C    P0             113W / 700W |  23197MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [7]:
%%bash

git clone https://github.com/NVIDIA/NeMo-Aligner.git
cd NeMo-Aligner && git checkout v0.2.0
pip install -e .

Cloning into 'NeMo-Aligner'...
Note: switching to 'v0.2.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 3b8b70e update package info


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///workspace/NeMo-Aligner
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/84

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
img2dataset 1.45.0 requires wandb<0.17,>=0.16.0, but you have wandb 0.15.3 which is incompatible.
img2dataset 1.45.0 requires webdataset<0.3,>=0.2.5, but you have webdataset 0.1.62 which is incompatible.
rapids-dask-dependency 23.12.1 requires dask==2023.11.0, but you have dask 2023.9.2 which is incompatible.
rapids-dask-dependency 23.12.1 requires distributed==2023.11.0, but you have distributed 2023.9.2 which is incompatible.
spacy 3.1.3 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
tb-nightly 2.17.0a20240220 requires tensorboard-data-server<0.8.0,>=0.7.0, but you have tensorboard-data-server 0.6.1 which is incompatible.
tensorrt-llm 0.7.1 requires transformers==4.33.1, but you have transformers 4.38.2 which is incompatible.
thinc 8.0.17 requires pydantic!=1.8,


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [34]:
%%bash
export MODEL_PATH=models/gemma-7b-sql-nemo
mkdir -p ${MODEL_PATH}

python NeMo-Aligner/examples/nlp/gpt/train_gpt_sft.py \
   name=gemma-7b-sql-nemo \
   trainer.precision=bf16 \
   trainer.num_nodes=1 \
   trainer.devices=8 \
   trainer.sft.max_steps=-1 \
   trainer.sft.limit_val_batches=40 \
   trainer.sft.val_check_interval=1000 \
   model.tensor_model_parallel_size=4 \
   model.pipeline_model_parallel_size=1 \
   model.megatron_amp_O2=True \
   model.restore_from_path=/workspace/models/pytorch-7b-pt.nemo \
   model.optim.lr=5e-6 \
   model.answer_only_loss=True \
   ++model.bias_activation_fusion=true \
   model.data.num_workers=0 \
   model.data.train_ds.micro_batch_size=1 \
   model.data.train_ds.global_batch_size=128 \
   model.data.train_ds.max_seq_length=8192 \
   model.data.train_ds.file_path=nsql.jsonl \
   model.data.validation_ds.micro_batch_size=1 \
   model.data.validation_ds.global_batch_size=128 \
   model.data.validation_ds.drop_last=True \
   model.data.validation_ds.file_path=nsql.jsonl \
   exp_manager.explicit_log_dir=${MODEL_PATH} \
   exp_manager.checkpoint_callback_params.save_nemo_on_train_end=True \
   exp_manager.resume_if_exists=True \
   exp_manager.resume_ignore_no_checkpoint=True \
   exp_manager.create_checkpoint_callback=True \
   exp_manager.checkpoint_callback_params.monitor=validation_loss

[NeMo W 2024-03-18 05:24:26 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    


[NeMo I 2024-03-18 05:24:26 train_gpt_sft:118] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-03-18 05:24:26 train_gpt_sft:119] 
    name: gemma-7b-sql-nemo
    trainer:
      num_nodes: 1
      devices: 8
      accelerator: gpu
      precision: bf16
      sft:
        max_epochs: 1
        max_steps: -1
        val_check_interval: 1000
        save_interval: ${.val_check_interval}
        limit_val_batches: 40
        gradient_clip_val: 1.0
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_time: null
      max_epochs: ${.sft.max_epochs}
      max_steps: ${.sft.max_steps}
    exp_manager:
      explicit_log_dir: models/gemma-7b-sql-nemo
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_p

[NeMo W 2024-03-18 05:24:26 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
      rank_zero_warn(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2024-03-18 05:24:26 exp_manager:708] Exp_manager is logging to models/gemma-7b-sql-nemo, but it already exists.
[NeMo W 2024-03-18 05:24:26 exp_manager:630] There were no checkpoints found in checkpoint_dir or no checkpoint folder at checkpoint_dir :models/gemma-7b-sql-nemo/checkpoints. Training from scratch.


[NeMo I 2024-03-18 05:24:26 exp_manager:396] Experiments will be logged at models/gemma-7b-sql-nemo
[NeMo I 2024-03-18 05:24:27 exp_manager:856] TensorboardLogger has been set up


[NeMo W 2024-03-18 05:24:54 megatron_base_model:1078] The model: GPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-18 05:24:54 megatron_base_model:1078] The model: GPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-18 05:24:54 megatron_base_model:1078] The model: GPTSFTModel() does not have field.name: tp_comm_overlap in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-18 05:24:54 megatron_base_model:1078] The model: GPTSFTModel() does not have field.name: tp_comm_split_ag in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-18 05:24:54 megatron_base_model:1078] The model: GPTSFTModel() does not have field.name: tp_comm_split_rs in its cfg. Add this key to cfg or config_ma

...

[NeMo W 2024-03-18 05:27:34 experimental:26] `<class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'>` is experimental and not ready for production yet. Use at your own risk.


[NeMo I 2024-03-18 05:27:40 megatron_gpt_model:1296] Pipeline model parallel rank: 0, Tensor model parallel rank: 0, Number of model parameters on device: 2.13e+09. Total number of model parameters: 8.54e+09.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
LOCAL_RANK: 4 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
LOCAL_RANK: 7 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
LOCAL_RANK: 5 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
LOCAL_RANK: 6 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
LOCAL_RANK: 2 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
LOCAL_RANK: 3 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


[NeMo I 2024-03-18 05:27:40 modelPT:723] Optimizer config = MegatronDistributedFusedAdam (
    Parameter Group 0
        betas: [0.9, 0.98]
        bias_correction: True
        eps: 1e-08
        lr: 5e-06
        weight_decay: 0.01
    
    Parameter Group 1
        betas: [0.9, 0.98]
        bias_correction: True
        eps: 1e-08
        lr: 5e-06
        weight_decay: 0.0
    )
[NeMo I 2024-03-18 05:27:40 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x793b24a42350>" 
    will be used during training (effective maximum steps = 613) - 
    Parameters : 
    (warmup_steps: 10
    constant_steps: 1000
    min_lr: 9.0e-07
    max_steps: 613
    )



  | Name  | Type          | Params
----------------------------------------
0 | model | Float16Module | 2.1 B 
----------------------------------------
2.1 B     Trainable params
0         Non-trainable params
2.1 B     Total params
8,538.206 Total estimated model params size (MB)
Training steps:   4%|▎         | 22/613 [05:05<1:58:33, 12.04s/it, train_grad_norm=9.06, train_lr=9e-7, train_loss=0.113, train_consumed_samples=2816, train_step_time=6.93, train_epoch=1]  

In [ ]:
! ls models/gemma-7b-sql-nemo

checkpoints
cmd-args.log
events.out.tfevents.1710737035.jupyter-pod.36463.0
hparams.yaml
lightning_logs.txt
nemo_error_log.txt
nemo_log_globalrank-0_localrank-0.txt
nemo_log_globalrank-1_localrank-1.txt
nemo_log_globalrank-2_localrank-2.txt
nemo_log_globalrank-3_localrank-3.txt
nemo_log_globalrank-4_localrank-4.txt
nemo_log_globalrank-5_localrank-5.txt
nemo_log_globalrank-6_localrank-6.txt
nemo_log_globalrank-7_localrank-7.txt
run_0


In [ ]:
! mkdir -p models/gemma-7b-sql-nemo-trt-llm

In [ ]:
from nemo.export import TensorRTLLM
trt_llm_exporter = TensorRTLLM(model_dir="models/gemma-7b-sql-nemo-trt-llm")
trt_llm_exporter.export(nemo_checkpoint_path="models/gemma-7b-sql-nemo/checkpoints/gemma-7b-sql-nemo.nemo", model_type="gemma", n_gpus=1)

Loaded mpi lib /usr/local/lib/python3.10/dist-packages/mpi4py/MPI.cpython-310-x86_64-linux-gnu.so successfully


saving weights: 100%|██████████| 169/169 [00:23<00:00,  7.26it/s] 


Before engine building, CPU RAM Used (GB): 18.153247833251953


[03/18/2024-03:25:28] [TRT-LLM] [W] Invalid timing cache, using freshly created one
[03/18/2024-03:25:28] [TRT-LLM] [I] Context FMHA Enabled


[03/18/2024-03:25:23] [TRT] [I] [MemUsageChange] Init CUDA: CPU +1, GPU +0, now: CPU 18275, GPU 3623 (MiB)
[03/18/2024-03:25:28] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +4397, GPU +1158, now: CPU 22808, GPU 4781 (MiB)
[03/18/2024-03:25:28] [TRT] [W] A shape layer can only run in INT32 precision
[03/18/2024-03:25:28] [TRT] [W] IElementWiseLayer with inputs LMHeadModelBuilder/vocab_embedding/GATHER_0_output_0 and LMHeadModelBuilder/layers/0/input_layernorm/SHUFFLE_0_output_0: first input has type BFloat16 but second input has type Float.
[03/18/2024-03:25:28] [TRT] [W] A shape layer can only run in INT32 precision
[03/18/2024-03:25:28] [TRT] [W] IElementWiseLayer with inputs LMHeadModelBuilder/layers/0/input_layernorm/REDUCE_AVG_0_output_0 and LMHeadModelBuilder/layers/0/input_layernorm/SHUFFLE_1_output_0: first input has type BFloat16 but second input has type Float.
[03/18/2024-03:25:28] [TRT] [W] A shape layer can only run in INT32 precision
[03/18/2024-03:25:28] [

...

[03/18/2024-03:26:20] [TRT] [I] [MemUsageStats] Peak memory usage during Engine building and serialization: CPU: 41498 MiB


[03/18/2024-03:26:41] [TRT-LLM] [I] Engine serialized. Total time: 00:00:21


[03/18/2024-03:26:41] [TRT] [I] Serialized 59 bytes of code generator cache.
[03/18/2024-03:26:41] [TRT] [I] Serialized 551917 bytes of compilation cache.
[03/18/2024-03:26:41] [TRT] [I] Serialized 49 timing cache entries


[03/18/2024-03:26:41] [TRT-LLM] [I] Timing cache serialized to models/gemma_dolly_finetuned_trt_llm/model.cache
[03/18/2024-03:26:42] [TRT-LLM] [I] Total time of building all 1 engines: 00:01:18
[03/18/2024-03:26:43] [TRT-LLM] [I] Reading from serialize path models/gemma_dolly_finetuned_trt_llm/NeMo_bfloat16_tp1_rank0.engine


After Engine building rank 0, CPU RAM Used (GB): 23.14820098876953
[03/18/2024-03:26:52] [TRT] [I] Loaded engine size: 17795 MiB
[03/18/2024-03:26:54] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +64, now: CPU 36079, GPU 21579 (MiB)
[03/18/2024-03:26:54] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +64, now: CPU 36079, GPU 21643 (MiB)
[03/18/2024-03:26:54] [TRT] [W] TensorRT was linked against cuDNN 8.9.6 but loaded cuDNN 8.9.4
[03/18/2024-03:26:54] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +17784, now: CPU 0, GPU 17784 (MiB)
[03/18/2024-03:26:54] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +64, now: CPU 36079, GPU 22277 (MiB)
[03/18/2024-03:26:54] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +64, now: CPU 36079, GPU 22341 (MiB)
[03/18/2024-03:26:54] [TRT] [W] TensorRT was linked against cuDNN 8.9.6 but loaded cuDNN 8.9.4
[03/18/2024-03:26:54] [TRT] [I] [MemUsageChange] TensorRT-managed all

In [ ]:
! ls models/models/gemma-7b-sql-nemo-trt-llm

NeMo_bfloat16_tp1_rank0.engine	config.json  model.cache  tokenizer.model


In [16]:
trt_llm_exporter.forward(["NVIDIA and Google are"])

[NeMo W 2024-03-18 04:04:38 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nested/__init__.py:165: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
      return _nested.nested_tensor(
    


[[' teaming up to bring the cloud gaming service, NVIDIA GeForce NOW, to Android TV. The service will be available on Android TV devices from Sony, TCL, and Xiaomi.\n\nNVIDIA GeForce NOW is a cloud gaming service that allows you to play PC games on your Android TV. The service is powered by NVIDIA’s cloud']]